In [8]:
from yf_utils import _2_split_train_val_test, _3_random_slices, _4_lookback_slices
from yf_utils import _5_perf_ranks, _6_grp_tuples_sort_sum
from myUtils import pickle_load

# verbose = False  # True prints more output
verbose = True  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
fp_df_close_clean = 'df_close_clean'

df_close_clean = pickle_load(path_data_dump, fp_df_close_clean)

In [9]:
# Split df_close_clean into training (df_train), validation (df_val) and test (df_test) set.
# The default split is 0.7, 0.2, 0.1 respectively.
df_train, df_val, df_test = _2_split_train_val_test(df_close_clean)
len_df_train = len(df_train)
len_df_val = len(df_val)
len_df_test = len(df_test)

In [10]:
# number of max lookback tuples to create for iloc start_train:end_train:end_eval
n_max_lb_slices = 4  
# n_max_lb_slices = 100

# for training, the number of days to lookback from max lookback end_train
days_lookbacks = [30, 120, 60]

# number of days from end_train are used to evaluate effectiveness of the training
days_eval = 10  

# set_top_syms = 5  # number of the most-common symbols from days_lookbacks' performance rankings
set_top_syms = 10  # number of the most-common symbols from days_lookbacks' performance rankings to keep

idx_eval_start = 0  #  start index of set_top_syms for evaluation
idx_eval_end = 4  #  end index of set_top_syms for evaluation

In [11]:
# create n_max_lb_slices of iloc start_train:end_train:end_eval using max value in days_loobacks
max_lookback_slices = _3_random_slices(len_df_train, n_max_lb_slices=n_max_lb_slices, days_lookback=max(days_lookbacks), days_eval=days_eval)
sets_lookback_slices = _4_lookback_slices(max_slices=max_lookback_slices, days_lookbacks=days_lookbacks, verbose=False)

if verbose:
  print(f'max_lookback_slices:\n{max_lookback_slices}')
  print(f'sets_lookback_slices:\n{sets_lookback_slices}')

max_lookback_slices:
[(818, 938, 948), (442, 562, 572), (1, 121, 131), (799, 919, 929)]
sets_lookback_slices:
[[(908, 938, 948), (878, 938, 948), (818, 938, 948)], [(532, 562, 572), (502, 562, 572), (442, 562, 572)], [(91, 121, 131), (61, 121, 131), (1, 121, 131)], [(889, 919, 929), (859, 919, 929), (799, 919, 929)]]


In [12]:
grp_top_set_syms_n_freq = []  # list of lists of top_set_symbols_n_freq, there are n_max_lb_slices lists in list
grp_top_set_syms = []  # list of lists of top_set_symbols, there are n_max_lb_slices lists in list
# loop thru lists of tuples of start_train:end_train:end_eval, i.e.
#  [[(887, 917, 927), (857, 917, 927), (797, 917, 927)],
#  [(483, 513, 523), (453, 513, 523), (393, 513, 523)]]
for lb_slices in sets_lookback_slices:
  grp_most_common_syms = []  
  for lb_slice in lb_slices:  # lb_slice, e.g. (246, 276, 286)
    start_train = lb_slice[0]
    end_train = lb_slice[1]
    start_eval = end_train
    end_eval = lb_slice[2]
    lookback = end_train - start_train
    eval = end_eval - start_eval

    if verbose:
      print(f'lb_slices:     {lb_slices}')
      print(f'lb_slice:      {lb_slice}')
      print(f'days lookback: {lookback}')
      print(f'days eval:     {eval}')    
      print(f'start_train:   {start_train}')
      print(f'end_train:     {end_train}')
      # print(f'start_eval:    {start_eval}')
      # print(f'end_eval:      {end_eval}')`

    _df = df_train.iloc[start_train:end_train]
    perf_ranks, most_common_syms = _5_perf_ranks(_df, n_top_syms=set_top_syms)
    grp_most_common_syms.append(most_common_syms)
    
    if verbose:    
      # 1 lookback of r_CAGR/UI, r_CAGR/retnStd, r_retnStd/UI
      print(f'perf_ranks: {perf_ranks}')  
      # most common symbols of perf_ranks 
      print(f'most_common_syms: {most_common_syms}')     
      # grp_perf_ranks[lookback] = perf_ranks
      print(f'+++ finish lookback slice {lookback} +++\n')

  if verbose:
    print(f'grp_most_common_syms: {grp_most_common_syms}')
    # grp_most_common_syms a is list of lists of tuples of 
    #  the most-common-symbols symbol:frequency cumulated from
    #  each days_lookback  
    print(f'**** finish lookback slices {lb_slices} ****\n')

  # flatten list of lists of (symbol:frequency)
  flat_grp_most_common_syms = [val for sublist in grp_most_common_syms for val in sublist]
  # group symbols from set of days_lookbacks (i.e. lb_slices) and sum frequency of the symbols
  set_most_common_syms = _6_grp_tuples_sort_sum(flat_grp_most_common_syms, reverse=True)
  # get the top few most-frequent symbol:frequency pairs
  top_set_syms_n_freq = set_most_common_syms[0:set_top_syms]
  # get symbols from top_set_syms_n_freq

###################################  
  # top_set_syms = [i[0] for i in top_set_syms_n_freq]
  top_set_syms = [i[0] for i in top_set_syms_n_freq[idx_eval_start:idx_eval_end]]  
###################################  
  
  grp_top_set_syms_n_freq.append(top_set_syms_n_freq)
  grp_top_set_syms.append(top_set_syms)

  if verbose:  
    print(f'top {set_top_syms} ranked symbols and frequency from set {lb_slices}:\n{top_set_syms_n_freq}')
    print(f'top {set_top_syms} ranked symbols from set {lb_slices}:\n{top_set_syms}')  
    print(f'===== finish top {set_top_syms} ranked symbols from days_lookback set {lb_slices} =====\n\n')

lb_slices:     [(908, 938, 948), (878, 938, 948), (818, 938, 948)]
lb_slice:      (908, 938, 948)
days lookback: 30
days eval:     10
start_train:   908
end_train:     938
perf_ranks: {'period-30': {'r_CAGR/UI': array(['DQ', 'NVCR', 'ENPH', 'ACRS', 'HIBB', 'APPS', 'GME', 'PCTY',
       'SGEN', 'TRUP'], dtype=object), 'r_CAGR/retnStd': array(['DQ', 'ACRS', 'NVCR', 'ENPH', 'HIBB', 'GME', 'APPS', 'OLN', 'TRUP',
       'SPWR'], dtype=object), 'r_retnStd/UI': array(['SGEN', 'EXAS', 'DQ', 'PCTY', 'PERI', 'BGNE', 'APPS', 'AAWW',
       'PKX', 'DG'], dtype=object)}}
most_common_syms: [('DQ', 3), ('APPS', 3), ('NVCR', 2), ('ENPH', 2), ('ACRS', 2), ('HIBB', 2), ('GME', 2), ('PCTY', 2), ('SGEN', 2), ('TRUP', 2), ('OLN', 1), ('SPWR', 1), ('EXAS', 1), ('PERI', 1), ('BGNE', 1), ('AAWW', 1), ('PKX', 1), ('DG', 1)]
+++ finish lookback slice 30 +++

lb_slices:     [(908, 938, 948), (878, 938, 948), (818, 938, 948)]
lb_slice:      (878, 938, 948)
days lookback: 60
days eval:     10
start_train:   878
en

In [13]:
print('z_grp_top_set_syms:')
z_grp_top_set_syms = zip(max_lookback_slices, grp_top_set_syms)
for item in z_grp_top_set_syms:
  print(item)
print('')

z_grp_top_set_syms:
((818, 938, 948), ['APPS', 'HIBB', 'DQ', 'GME'])
((442, 562, 572), ['FTSM', 'JYNT', 'CPRX', 'QCOM'])
((1, 121, 131), ['AVEO', 'BGNE', 'BLFS', 'CPRX'])
((799, 919, 929), ['HIBB', 'GOGO', 'RUN', 'APPS'])



In [14]:
from yf_utils import _7_perf_eval
print('z_grp_top_set_syms:')
z_grp_top_set_syms = zip(max_lookback_slices, grp_top_set_syms)
# z_grp_top_set_syms = zip(max_lookback_slices, grp_top_set_syms[26:29])

for i, (_lookback_slice, _top_set_syms) in enumerate(z_grp_top_set_syms):
# for i, (_lookback_slice, _top_set_syms) in enumerate(z_grp_top_set_syms[26:29]):

  print(f'{i + 1 } of {n_max_lb_slices} max_lookback_slice')
  print(f'max_lookback_slice: {_lookback_slice}')
  # dates correspond to max_lookback_slice
  _date_0 = df_train.index[_lookback_slice[0]].strftime('%Y-%m-%d')
  _date_1 = df_train.index[_lookback_slice[1]].strftime('%Y-%m-%d')
  _date_2 = df_train.index[_lookback_slice[2]].strftime('%Y-%m-%d')
  print(f'max lookback dates: {_date_0}, {_date_1}, {_date_2}')  
  print(f'top_set_syms: {_top_set_syms}\n')

  start_eval = _lookback_slice[1]
  end_eval = _lookback_slice[2]
  df_eval = df_train[start_eval:end_eval][_top_set_syms]

  if verbose:
    print(f'start_eval: {start_eval}')
    print(f'end_eval:   {end_eval}')  
    print(f'\ndf_eval:\n{df_eval}\n')


  _, grp_retnStd_d_UI, grp_CAGR_d_retnStd, grp_CAGR_d_UI = _7_perf_eval(df_eval)
  print(f'grp(retnStd/UI):   mean, std, mean/std: {grp_retnStd_d_UI[0]:>13,.3f}, {grp_retnStd_d_UI[1]:>13,.3f}, {grp_retnStd_d_UI[2]:>13,.3f}')
  print(f'grp(CAGR/retnStd): mean, std, mean/std: {grp_CAGR_d_retnStd[0]:>13,.3f}, {grp_CAGR_d_retnStd[1]:>13,.3f}, {grp_CAGR_d_retnStd[2]:>13,.3f}')
  print(f'grp(CAGR/UI):      mean, std, mean/std: {grp_CAGR_d_UI[0]:>13,.3f}, {grp_CAGR_d_UI[1]:>13,.3f}, {grp_CAGR_d_UI[2]:>13,.3f}')

  _sym_idx = ['SPY']
  df_SPY = df_train[start_eval:end_eval][_sym_idx]
  _, grp_retnStd_d_UI, grp_CAGR_d_retnStd, grp_CAGR_d_UI = _7_perf_eval(df_SPY)
  # print(f'\n_sym_idx: {_sym_idx}')
  # print(f'grp(retnStd/UI):   mean, std, mean/std: {grp_retnStd_d_UI[0]:>13,.3f}, {grp_retnStd_d_UI[1]:>13,.3f}, {grp_retnStd_d_UI[2]:>13,.3f}')
  # print(f'grp(CAGR/retnStd): mean, std, mean/std: {grp_CAGR_d_retnStd[0]:>13,.3f}, {grp_CAGR_d_retnStd[1]:>13,.3f}, {grp_CAGR_d_retnStd[2]:>13,.3f}')
  # print(f'grp(CAGR/UI):      mean, std, mean/std: {grp_CAGR_d_UI[0]:>13,.3f}, {grp_CAGR_d_UI[1]:>13,.3f}, {grp_CAGR_d_UI[2]:>13,.3f}')
  print(f'\nSPY: retnStd/UI, CAGR/retnStd, CAGR/UI: {grp_retnStd_d_UI[0]:>13,.3f}, {grp_CAGR_d_retnStd[0]:>13,.3f}, {grp_CAGR_d_UI[0]:>13,.3f}')
  print('='*50, '\n')  

z_grp_top_set_syms:
1 of 4 max_lookback_slice
max_lookback_slice: (818, 938, 948)
max lookback dates: 2020-04-28, 2020-10-16, 2020-10-30
top_set_syms: ['APPS', 'HIBB', 'DQ', 'GME']

start_eval: 938
end_eval:   948

df_eval:
                 APPS       HIBB         DQ     GME
Date                                               
2020-10-16  38.389999  48.685032  40.001999  3.3275
2020-10-19  36.430000  49.642380  42.228001  3.4775
2020-10-20  36.099998  49.672298  48.397999  3.4650
2020-10-21  34.209999  47.558155  39.821999  3.5250
2020-10-22  32.860001  46.231834  39.599998  3.7275
2020-10-23  33.849998  44.087776  39.618000  3.7500
2020-10-26  31.709999  42.821289  35.762001  3.3625
2020-10-27  31.490000  42.791370  34.778000  3.1725
2020-10-28  30.030001  40.258392  35.875999  2.9550
2020-10-29  31.879999  39.121544  37.537998  2.9325

grp(retnStd/UI):   mean, std, mean/std:         0.372,         0.132,         2.807
grp(CAGR/retnStd): mean, std, mean/std:       -22.236,        12.78